## Packages

In [1]:
from pyomo.environ import *
import numpy as np
import random
import pandas as pd
from numpy import flatnonzero as find
from pypower.api import case9, ext2int, bustypes, makeBdc, rundcopf, ppoption
from pypower.idx_bus import BUS_TYPE, REF, VA, PD, LAM_P, LAM_Q, MU_VMAX, MU_VMIN
from pypower.idx_gen import PG, MU_PMAX, MU_PMIN, MU_QMAX, MU_QMIN, PMAX, PMIN, GEN_BUS
from pypower.idx_brch import PF, PT, QF, QT, RATE_A, MU_SF, MU_ST
from pypower.idx_cost import COST
import matplotlib.pyplot as plt
import pickle

## Power network parameters

In [2]:
# Run PyPower case to get initial condition
ppc = case9()
ppopt = ppoption(VERBOSE=0)
pp_dcopf = rundcopf(ppc, ppopt)
pp_bus = pp_dcopf["bus"]
pp_branch = pp_dcopf["branch"]
pp_gen = pp_dcopf["gen"]


Converged in 0.04 seconds
Objective Function Value = 5216.03 $/hr
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     820.0           0.0 to 0.0
Generators         3     On-line Capacity       820.0           0.0 to 0.0
Committed Gens     3     Generation (actual)    315.0               0.0
Loads              3     Load                   315.0               0.0
  Fixed            3       Fixed                315.0               0.0
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             0     Shunt (inj)              0.0               0.0
Branches           9     Losses (I^2 * Z)         0.00              0.00
Transformers       0     Branch Charging (inj)     -                0.0
Inter-ties         0     Total Inter-tie F

In [15]:
## convert to internal indexing
ppc = ext2int(ppc)
baseMVA, bus, gen, branch, gencost = \
    ppc["baseMVA"], ppc["bus"], ppc["gen"], ppc["branch"], ppc["gencost"]

## get bus index lists of each type of bus
ref, pv, pq = bustypes(bus, gen)
pvpq = np.matrix(np.r_[pv, pq])

## generator info
gbus = gen[:, GEN_BUS].astype(int)    ## what buses are they at?
refgen = find(gbus == ref)

## build B matrices and phase shift injections
B, Bf, _, _ = makeBdc(baseMVA, bus, branch)
B = B.todense()
Bf = Bf.todense()

# Problem dimensions
NG = gen.shape[0] # Number of generators
NB = bus.shape[0] # Number of buses
NBr = branch.shape[0] # Number of lines
NL = np.count_nonzero(pp_bus[:, PD]) # Number of loads
NW = 1 # Number of wind farms

# Map generator to bus
CG = np.zeros((NB,NG))
CG[gbus,range(3)] = 1

# Map load to bus
CL = np.zeros((NB,NL))
load_bus = np.nonzero(pp_bus[:, PD])[0]
for i in range(len(load_bus)):
    CL[load_bus[i], i] = 1

# Map wind farm to bus
CW = np.zeros((NB,1))
CW[5,0] = 1

# Generator capacity limit p.u.
Pmax = gen[:, PMAX]/baseMVA
Pmin = gen[:, PMIN]/baseMVA

# Line flow limit p.u.
Lmax = branch[:, RATE_A]/baseMVA
Lmin = - branch[:, RATE_A]/baseMVA

# Quadratic cost coefficients
# Convert to p.u.
CG2 = gencost[:, COST]*baseMVA**2
CG1 = gencost[:, COST+1]*baseMVA
CG0 = gencost[:, COST+2]

# Cost coefficient for wind power
CW1 = 0
CW0 = 0

# Cost coefficients for generation reserve
CRG_up = 10*np.ones((NG,1))*baseMVA
CRG_down = 10*np.ones((NG,1))*baseMVA

# Cost coefficients for load reserve
CRL_up = 9.8*np.ones((NL,1))*baseMVA
CRL_down = 9.8*np.ones((NL,1))*baseMVA

# Uncontrollable load
load = pp_bus[:,PD]
PL_total = load[load > 0]
PL = PL_total/baseMVA

# Generator and demand set points
# Power in p.u., Va in rad
Pg0 = pp_gen[:, PG]/baseMVA
Pd0 = pp_bus[:, PD]/baseMVA
Pinj0 = (np.matmul(CG, Pg0) - Pd0)
Va0 = pp_bus[:, VA]/180*np.pi

## Read temperature and wind data

In [26]:
# Read temperature data
weather_data = pd.read_csv("Weather_Data/NYC_Hourly_Normal_Temp.csv")
# Read wind power data
PW_data = pd.read_csv("spring_wind.csv",header=None).to_numpy()
PW_data = PW_data/500*20/baseMVA
# PW_data = PW_data[:, 0:4000]

In [27]:
Tamb = (weather_data.iloc[:,8] - 32)/1.8
Tamb_min = min(Tamb)
Tamb_10PCTL = (weather_data.iloc[:,6] - 32)/1.8
Tamb_90PCTL = (weather_data.iloc[:,7] - 32)/1.8
std_Tamb_10PCTL = (Tamb - Tamb_10PCTL)/1.282 
std_Tamb_90PCTL = (Tamb_90PCTL - Tamb)/1.282 
std_Tamb = (std_Tamb_10PCTL + std_Tamb_90PCTL)/10

## Virtual storage model

In [28]:
def VS(dt,Tamb,num_TCL,Tset,delta,R,C,P_el,COP,baseMVA):
    Tdown = np.zeros(num_TCL)
    Tup = np.zeros(num_TCL)
    T_ON = np.zeros(num_TCL)
    T_OFF = np.zeros(num_TCL)
    D = np.zeros(num_TCL)
    B = 0
    Pdown = 0
    Pup = 0
    Sdown = 0
    Sup = 0
    for i in range(0,num_TCL):
        Tdown[i] = Tset[i] - delta/2
        Tup[i] = Tset[i] + delta/2
        T_ON[i] = C[i]*R[i]*np.log((Tdown[i] - Tamb - R[i]*P_el[i]*COP[i])/(Tup[i] - Tamb - R[i]*P_el[i]*COP[i]))
        if ((Tdown[i] - Tamb - R[i]*P_el[i]*COP[i])/(Tup[i] - Tamb - R[i]*P_el[i]*COP[i]) < 0):
            print(Tdown[i] - Tamb - R[i]*P_el[i]*COP[i])
            print(Tup[i] - Tamb - R[i]*P_el[i]*COP[i])
        T_OFF[i] = C[i]*R[i]*np.log((Tup[i] - Tamb)/(Tdown[i] - Tamb))
        D[i] = T_ON[i]/(T_ON[i] + T_OFF[i])
        if ( Tamb < Tdown[i] and Tamb + R[i]*P_el[i]*COP[i] > Tup[i] ):
            B = B + P_el[i]*D[i]
            Pup = Pup + P_el[i]
            Sup = Sup + dt*P_el[i]*(1 - D[i])
    return [B/1000/baseMVA,Pdown/1000/baseMVA,Pup/1000/baseMVA,Sdown,Sup]

## Optimization problem

In [82]:
# Optimization: Problem formulation
def optimizer(NG,NL,Tamb_sample1,Tamb_sample2,PW_sample,num_TCL,Tset,delta,R,C,P_el,COP,S0,PL,\
    Pmax,Pmin,Lmax,Lmin,CG,CW,CL,CG0,CG1,CG2,CW0,CW1,CRG_up,CRG_down,CRL_up,CRL_down,nsample,PW_f):
    model = ConcreteModel()
    # Defining the variables
    model.PG = Var(range(NG),within = NonNegativeReals, initialize=0)
    model.PC = Var(range(NL),within = NonNegativeReals, initialize=0)
    model.RG_down = Var(range(NG),within = NonNegativeReals, initialize=0)
    model.RG_up = Var(range(NG),within = NonNegativeReals, initialize=0)
    model.RG = Var(range(NG),range(nsample),within = Reals, initialize=0)
    # model.RL_down = Var(range(NL),within = NonNegativeReals, initialize=0)
    # model.RL_up = Var(range(NL),within = NonNegativeReals, initialize=0)
    # model.RL = Var(range(NL),range(nsample),within = Reals, initialize=0)
    model.dG_down = Var(range(NG),within = NonNegativeReals, initialize=0)
    model.dG_up = Var(range(NG),within = NonNegativeReals, initialize=0)
    # model.dL_down = Var(range(NL),within = NonNegativeReals, initialize=0)
    # model.dL_up = Var(range(NL),within = NonNegativeReals, initialize=0)
    model.Va = Var(range(NB), within=Reals, initialize=0, bounds=(-1,1))
    model.s1 = Var(range(nsample), within=NonNegativeReals, initialize=0)
    model.s2 = Var(range(nsample), within=NonNegativeReals, initialize=0)
    model.s3 = Var(range(nsample), within=NonNegativeReals, initialize=0)
    model.s4 = Var(range(nsample), within=NonNegativeReals, initialize=0)
    model.s5 = Var(range(nsample), within=NonNegativeReals, initialize=0)
    model.s6 = Var(range(nsample), within=NonNegativeReals, initialize=0)
    # model.s7 = Var(range(nsample), within=NonNegativeReals, initialize=0)
    # model.s8 = Var(range(nsample), within=NonNegativeReals, initialize=0)
    model.d = Var(range(nsample), within=NonNegativeReals, initialize=0)
    model.e = Var(range(nsample), within=NonNegativeReals, initialize=0)
    model.dual = Suffix(direction=Suffix.IMPORT)

    ## Defining the deterministic constraints
    model.c = ConstraintList()

    # Power mismatch distribution
    model.c.add(sum(model.dG_down[i] for i in range(NG)) == 1)
    model.c.add(sum(model.dG_up[i] for i in range(NG)) == 1)
    
    # Line flow limit constraints
    for i in range(NBr):
        model.c.add(sum(Bf[i,j]*model.Va[j] for j in range(NB)) <= Lmax[i])
        model.c.add(sum(Bf[i,j]*model.Va[j] for j in range(NB)) >= Lmin[i])
    
    PB_all = np.zeros((NL,nsample))
    P_mis_wind_all = np.zeros(nsample)
    identified_VS_all = np.zeros((NL,nsample,5))
    
    # Defining the CC constraints
    for i in range(nsample):

        for j in range(NG):
        # Generation capacity limit
            model.c.add(model.PG[j] + model.RG[j,i] >= Pmin[j])
            model.c.add(model.PG[j] + model.RG[j,i] <= Pmax[j])
            model.c.add(model.PG[j] + model.RG_up[j] <= Pmax[j])
            model.c.add(model.PG[j] - model.RG_down[j] >= Pmin[j])
            # Reserve capacity limit
            model.c.add(model.RG[j,i] >= -model.RG_down[j])
            model.c.add(model.RG[j,i] <= model.RG_up[j])
    
        # DC Power flow constraint
        for j in range(NB):
            model.c.add(sum(CG[j,k]*model.PG[k] + CG[j,k]*model.RG[k,i] for k in range(NG)) + CW[j]*PW_sample[i]\
            - sum(CL[j,k]*PL[k] + CL[j,k]*model.PC[k] for k in range(NL)) >= sum(B[j,k]*model.Va[k] for k in range(NB)) - model.s1[i])
            model.c.add(sum(CG[j,k]*model.PG[k] + CG[j,k]*model.RG[k,i] for k in range(NG)) + CW[j]*PW_sample[i]\
            - sum(CL[j,k]*PL[k] + CL[j,k]*model.PC[k] for k in range(NL)) <= sum(B[j,k]*model.Va[k] for k in range(NB)) + model.s2[i])
            # model.c.add(sum(CG[j,k]*model.PG[k] + CG[j,k]*model.RG[k,i] for k in range(NG)) + CW[j]*PW_sample[i]\
            # - sum(CL[j,k]*PL[k] + CL[j,k]*model.PC[k] for k in range(NL)) >= sum(B[j,k]*model.Va[k] for k in range(NB)) - model.s1[i])
            # model.c.add(sum(CG[j,k]*model.PG[k] + CG[j,k]*model.RG[k,i] for k in range(NG)) + CW[j]*PW_sample[i]\
            # - sum(CL[j,k]*PL[k] + CL[j,k]*model.PC[k] for k in range(NL)) <= sum(B[j,k]*model.Va[k] for k in range(NB)) + model.s2[i])
        
        P_mis_wind = PW_sample[i] - PW_f
        P_mis_wind_all[i] = P_mis_wind

        PB = np.zeros(NL)

        # print(i)

        # P_mis_load = 0
        for j in range(NL):
            identified_VS = VS(0.25,Tamb_sample1[i],num_TCL,Tset[:,j],delta,R[:,j],C[:,j],P_el[:,j],COP[:,j],baseMVA)
            # print("VS0", j, identified_VS[0])
            identified_VS_all[j,i,:] = identified_VS
            PB[j] = identified_VS[0]
        
        PB_all[:,i] = PB
        
        model.c.add(model.e[i] >= P_mis_wind - sum(model.PC[j] for j in range(NL)) + sum(PB[j] for j in range(NL)))
        model.c.add(model.d[i] >= - P_mis_wind + sum(model.PC[j] for j in range(NL)) - sum(PB[j] for j in range(NL)))
        
        for j in range(NG):
            model.c.add(model.RG[j,i] == model.dG_up[j]*model.d[i] - model.dG_down[j]*model.e[i])
        

        for j in range(NL):
            # model.c.add(model.RL[j,i] == model.dL_up[j]*model.e[i] - model.dL_down[j]*model.d[i])
            identified_VS1 = VS(0.25,Tamb_sample1[i],num_TCL,Tset[:,j],delta,R[:,j],C[:,j],P_el[:,j],COP[:,j],baseMVA)
            # identified_VS2 = VS(0.25,Tamb_sample2[i],num_TCL,Tset[:,j],delta,R[:,j],C[:,j],P_el[:,j],COP[:,j],baseMVA)

            model.c.add(model.PC[j] >= identified_VS1[1])
            model.c.add(model.PC[j] <= identified_VS1[2])
            # model.c.add(model.RL[j,i] >= -model.RL_down[j])
            # model.c.add(model.RL[j,i] <= model.RL_up[j])
            model.c.add(S0[j] + 0.25*(model.PC[j] - identified_VS1[0])*1000*baseMVA >= identified_VS1[3] - model.s3[i])
            model.c.add(S0[j] + 0.25*(model.PC[j] - identified_VS1[0])*1000*baseMVA  <= identified_VS1[4] + model.s4[i])
            model.c.add(S0[j] + 0.75*(model.PC[j] - identified_VS1[0])*1000*baseMVA  + 0.25*(model.PC[j]\
                 - identified_VS1[0])*1000*baseMVA  >= identified_VS1[3] - model.s5[i])
            model.c.add(S0[j] + 0.75*(model.PC[j] - identified_VS1[0])*1000*baseMVA  + 0.25*(model.PC[j]\
                 - identified_VS1[0])*1000*baseMVA  <= identified_VS1[4] + model.s6[i])

            # model.c.add(S0[j] + 0.75*(model.PC[j] - identified_VS2[0])*1000*baseMVA  + 0.25*(model.PC[j] + model.RL[j,i]\
            #      - identified_VS2[0])*1000*baseMVA  >= identified_VS2[3] - model.s7[i])
            # model.c.add(S0[j] + 0.75*(model.PC[j] - identified_VS2[0])*1000*baseMVA  + 0.25*(model.PC[j] + model.RL[j,i]\
            #      - identified_VS2[0])*1000*baseMVA  <= identified_VS2[4] + model.s8[i])
    
    def cost(model, CG0, CG1, CG2, CRG_down, CRG_up, nsample, CRL_down, CRL_up):

        cost = sum(CG0[i] for i in range(NG))\
                + sum(CG1[i]*model.PG[i] for i in range(NG))\
                + sum(CG2[i]*model.PG[i]**2 for i in range(NG))\
                + sum(CRG_up[i]*model.RG_up[i] for i in range(NG))\
                + sum(CRG_down[i]*model.RG_down[i] for i in range(NG))\
                + 1000*sum(model.s1[i] + model.s2[i] for i in range(nsample))\
                + 1e10*sum(model.s3[i] + model.s4[i] + model.s5[i] + model.s6[i] for i in range(nsample))\
                + 1000*sum(model.d[i] + model.e[i] for i in range(nsample))
                # + sum(CRL_up[i]*model.RL_up[i] for i in range(NL))\
                # + sum(CRL_down[i]*model.RL_down[i] for i in range(NL))\
                # + 1e10*sum(model.s7[i] + model.s8[i] for i in range(nsample))\
                
                
        return cost

    model.obj = Objective(expr=cost(model, CG0, CG1, CG2, CRG_down, CRG_up, nsample, CRL_down, CRL_up), sense=minimize)

    
    return model, PB_all, P_mis_wind_all, identified_VS_all

## Load parameters

In [84]:
# Parameters related to the demand side
num_TCL = 5000
Tset = np.zeros((num_TCL,NL))
R = np.zeros((num_TCL,NL))
C = np.zeros((num_TCL,NL))
COP = np.zeros((num_TCL,NL))
P_el = np.zeros((num_TCL,NL))
Tset_set = [21,21.5,22,22.5,23]
for i in range(num_TCL): 
    for j in range(NL):
        s = random.sample(Tset_set,1)
        Tset[i,j] = s[0] 
delta = 1
R_nom = 2
R_low = 0.8*R_nom
R_high = 1.2*R_nom
C_nom = 2
C_low = 0.8*C_nom
C_high = 1.2*C_nom
COP_nom = 2.5
COP_low = 0.8*COP_nom
COP_high = 1.2*COP_nom

np.random.seed(1)

for i in range(NL): 
    R[:,i] = np.random.uniform(R_low,R_high,num_TCL)
    C[:,i] = np.random.uniform(C_low,C_high,num_TCL)
    COP[:,i] = np.random.uniform(COP_low,COP_high,num_TCL)
    for j in range(num_TCL):
        P_el[j,i] = np.ceil((Tset[j,i] + delta/2 - Tamb_min)/(R[j,i]*COP[j,i])) + 3

# Sampling
eps = 0.1
beta = 0.1
num_decision_variables = 5*NG + 5*NL
num_samples1 = np.ceil((2/eps)*(np.log(1/beta) + num_decision_variables)).astype(int)
num_samples2 = np.ceil((num_decision_variables + 1 + np.log(1/beta) + np.sqrt(2*(num_decision_variables+1)*np.log(1/beta)))/eps).astype(int)

print("Number of samples 1: ", num_samples1)
print("Number of samples 2: ", num_samples2)

P_B = np.zeros((NL,1))
Pdown = np.zeros((NL,1))
Pup = np.zeros((NL,1))
Sdown = np.zeros((NL,1))
Sup = np.zeros((NL,1))
S0 = np.zeros((NL,1))

for j in range(NL):
    [P_B[j],Pdown[j],Pup[j],Sdown[j],Sup[j]] = \
        VS(0.25,Tamb[0],num_TCL,Tset[:,j],delta,R[:,j],C[:,j],P_el[:,j],COP[:,j],baseMVA)
    S0[j] = (Sdown[j] + Sup[j])/2    


Number of samples 1:  647
Number of samples 2:  453


## Single period problem

In [85]:
# # Single period

# # Define the model
# nsample= 50
# np.random.seed(1)
# Tamb_sample = np.random.normal(Tamb[0],std_Tamb[0],nsample)
# PW_sample = np.random.choice(PW_data[0,:], size=nsample)
# PW_f = np.median(PW_data[0,:])
# # PW_f = np.median(PW_data)
# model, PB_all, P_mis_wind, identified_VS_all = optimizer(NG,NL,
# Tamb_sample,PW_sample,num_TCL,Tset,delta,R,C,P_el,COP,S0,PL,\
#     Pmax,Pmin,Lmax,Lmin,CG,CW,CL,CG0,CG1,CG2,CW0,CW1,CRG_up,CRG_down,CRL_up,CRL_down,nsample,PW_f)

# # Solve the problem
# solver = SolverFactory("bonmin")
# solver.solve(model)

# model.display()

## Multi-period problem

In [86]:
# Read S0 data from OPF with DR
# file = open("Results/S0all_453","rb")
# S0_all_DR = pickle.load(file)
# file.close()

In [107]:
# Multi-period 
import time

# S0_all = S0_all_DR
S0_all = np.zeros((24,NL))
R_PC = np.zeros((24,NL))
models = []
Tamb_samples1 = []
Tamb_samples2 = []
PW_samples = []
PW_fs = []
PB_alls = []
P_mis_wind_alls = []
identified_VS_alls = []
P_B_test = []
Pdown_test = []
Pup_test = []
Sdown_test = []
Sup_test = []

# Initialize at hour 0
for j in range(NL):
    [P_B[j],Pdown[j],Pup[j],Sdown[j],Sup[j]] = \
        VS(0.25,Tamb[0],num_TCL,Tset[:,j],delta,R[:,j],C[:,j],P_el[:,j],COP[:,j],baseMVA)
    S0_all[0,j] = (Sdown[j] + Sup[j])/2  

nsample= 453

for h in range(24):

    print("Hour ", h)
    t_start = time.time()

    print("S0: ", S0_all[h])

    np.random.seed(1)
    Tamb_sample1 = np.random.normal(Tamb[h],std_Tamb[h],nsample)
    Tamb_sample2 = np.random.normal(Tamb[h+1],std_Tamb[h+1],nsample)
    PW_sample = np.random.choice(PW_data[h,:], size=nsample)
    PW_f = np.median(PW_data[h, :])

    # Define the problem
    model, PB_all, P_mis_wind_all, identified_VS_all = optimizer(NG,NL,Tamb_sample1,Tamb_sample2,PW_sample,num_TCL,Tset,delta,R,C,P_el,COP,S0_all[h,:],PL,\
        Pmax,Pmin,Lmax,Lmin,CG,CW,CL,CG0,CG1,CG2,CW0,CW1,CRG_up,CRG_down,CRL_up,CRL_down,nsample,PW_f)

    # Solve the problem
    solver = SolverFactory("bonmin")
    solver.solve(model)
    r_pc = model.PC[:]()
    R_PC[h,:] = r_pc

    models.append(model)
    Tamb_samples1.append(Tamb_sample1)
    PW_samples.append(PW_sample)
    PW_fs.append(PW_f)
    PB_alls.append(PB_all)
    P_mis_wind_alls.append(P_mis_wind_all)
    identified_VS_alls.append(identified_VS_all)

    if h != 23:
        for j in range(NL):
            [P_B[j],Pdown[j],Pup[j],Sdown[j],Sup[j]] = \
                VS(0.25,Tamb[h],num_TCL,Tset[:,j],delta,R[:,j],C[:,j],P_el[:,j],COP[:,j],baseMVA)
            S0_all[h+1,j] = S0_all[h,j] + r_pc[j]*1000*baseMVA - P_B[j]*1000*baseMVA
        
            print("S up %d: %f" %(j, Sup[j]))

    t_end = time.time()
    print("Elapsed: ", t_end - t_start)


Hour  0
S0:  [2794.48257109 2798.6107005  2790.34622915]
S up 0: 5588.965142
S up 1: 5597.221401
S up 2: 5580.692458
Elapsed:  119.62601232528687
Hour  1
S0:  [2993.72054089 2999.19363934 3000.11664782]
S up 0: 5546.203297
S up 1: 5554.381374
S up 2: 5537.839264
Elapsed:  158.44917559623718
Hour  2
S0:  [3094.78089704 3113.02737514 3100.88079848]
S up 0: 5503.441452
S up 1: 5511.541347
S up 2: 5494.986069
Elapsed:  135.3354456424713
Hour  3
S0:  [3123.59019831 3126.96765805 3110.50478846]
S up 0: 5460.679600
S up 1: 5468.701315
S up 2: 5452.132865
Elapsed:  119.95629453659058
Hour  4
S0:  [3034.36016865 3037.94551439 3020.63056531]
S up 0: 5432.171692
S up 1: 5440.141287
S up 2: 5423.564056
Elapsed:  119.33100700378418
Hour  5
S0:  [3005.85400579 3009.38726516 2992.06347762]
S up 0: 5417.917735
S up 1: 5425.861270
S up 2: 5409.279648
Elapsed:  187.31128478050232
Hour  6
S0:  [3001.5076641  3005.01488006 2987.68690256]
S up 0: 5417.917735
S up 1: 5425.861270
S up 2: 5409.279648
Elapsed:

In [112]:
model.PC.pprint()

PC : Size=3, Index=PC_index
    Key : Lower : Value               : Upper : Fixed : Stale : Domain
      0 :     0 : 0.21552930057504951 :  None : False : False : NonNegativeReals
      1 :     0 :  0.2156447374589551 :  None : False : False : NonNegativeReals
      2 :     0 :  0.2158650084265998 :  None : False : False : NonNegativeReals


In [113]:
models[2].PG.pprint()

PG : Size=3, Index=PG_index
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
      0 :     0 : 1.0714170284240991 :  None : False : False : NonNegativeReals
      1 :     0 : 1.5467279062987633 :  None : False : False : NonNegativeReals
      2 :     0 : 1.1262174757106598 :  None : False : False : NonNegativeReals


In [114]:
model.RG_up.pprint()
model.RG_down.pprint()

RG_up : Size=3, Index=RG_up_index
    Key : Lower : Value                : Upper : Fixed : Stale : Domain
      0 :     0 : 0.034175305559122404 :  None : False : False : NonNegativeReals
      1 :     0 :  0.03417530672466326 :  None : False : False : NonNegativeReals
      2 :     0 :  0.03417530507559341 :  None : False : False : NonNegativeReals
RG_down : Size=3, Index=RG_down_index
    Key : Lower : Value                : Upper : Fixed : Stale : Domain
      0 :     0 :  0.05764701072062478 :  None : False : False : NonNegativeReals
      1 :     0 : 0.057647004903107125 :  None : False : False : NonNegativeReals
      2 :     0 : 0.057647010348583425 :  None : False : False : NonNegativeReals


## Save the results

In [111]:
import pickle

case_name = "case9_without_DR_v6"

# Write models
file = open("Results/models_%s_%d" %(case_name,nsample), "wb")
pickle.dump(models, file)
file.close()

# Write S0_all
file = open("Results/S0all_%s_%d" %(case_name,nsample),"wb")
pickle.dump(S0_all, file)
file.close()

# Write R_PC
file = open("Results/R_PC_%s_%d" %(case_name,nsample), "wb")
pickle.dump(R_PC, file)
file.close()

# Write Tamb_samples1
file = open("Results/Tamb_samples1_%s_%d" %(case_name,nsample), "wb")
pickle.dump(Tamb_samples1, file)
file.close()

# Write Tamb_samples2
file = open("Results/Tamb_samples2_%s_%d" %(case_name,nsample), "wb")
pickle.dump(Tamb_samples2, file)
file.close()

# Write PW_samples
file = open("Results/PW_samples_%s_%d" %(case_name,nsample), "wb")
pickle.dump(PW_samples, file)
file.close()

# Write PW_fs
file = open("Results/PW_fs_%s_%d" %(case_name,nsample), "wb")
pickle.dump(PW_fs, file)
file.close()

# Write PB
file = open("Results/PB_alls_%s_%d" %(case_name,nsample), "wb")
pickle.dump(PB_alls, file)
file.close()

# Write identified_VS_alls
file = open("Results/VS_alls_%s_%d" %(case_name,nsample), "wb")
pickle.dump(identified_VS_alls, file)
file.close()

# Write Pmis_winds
file = open("Results/P_mis_winds_%s_%d" %(case_name,nsample), "wb")
pickle.dump(P_mis_wind_alls, file)
file.close()